In [2]:
import re
import time
import datetime
import requests
import pandas as pd
import multiprocessing
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from Script.Function.funciones import movies_for_category, categories, print_process, movies, series_informacion, desempaquetado, generador_dummies

# -------------- Parametros --------------
ruta_base = r"Bases_datos/"

# ----------------------------------------

In [2]:
print_process("Comienzo del Notebook 'Scraping'")

|-----------------------Comienzo del Notebook 'Scraping'-----------------------|


In [3]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 27.2 segundos-----------------------|


In [4]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 26.86 segundos|
|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 27.4 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 28.65 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 29.04 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 29.49 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 29.84 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 31.83 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 32.73 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 42.46 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 14.35 segundos|
|Todo el contenido de la categ

In [5]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [6]:
print_process("Comienzo del proceso de extraccion de informacion de cada iteracion")

|-----Comienzo del proceso de extraccion de informacion de cada iteracion-----|


In [7]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies) / 60,2)} minutos")


Error al cargar la url https://filmzie.com/content/the-wedding en el intento 1: 404


In [ ]:
df = pd.DataFrame(todo)

In [ ]:

df.to_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [3]:
df = pd.read_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv")

In [4]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [5]:

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)
peliculas.loc[peliculas["idioma"] == 'Non linguistic content', "idioma"] = "No contiene idioma"
peliculas["subtitulos"] = peliculas["subtitulos"].fillna("No contiene subtitulos")


In [6]:
peliculas.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,categorias,sinopsis,tipo,link
0,Inappropriate,Italian,English,2020,23,"Short, Drama",Experience the extraordinary love between two ...,Pelicula,https://filmzie.com/content/inappropriate-2020
1,Peace Pox,English,English,2011,67,"History, War","Inspired by the Christmas Truce of 1914, soldi...",Pelicula,https://filmzie.com/content/peace-pox-2011
2,The Dark Web,English,"English, Spanish, Albanian",2020,27,"Thriller, Short",How far will a father go to save his son?,Pelicula,https://filmzie.com/content/the-dark-web-2020
3,Don't Call Me Bigfoot,English,"English, Spanish",2020,72,Documentary,"There are tales all around the world of big, h...",Pelicula,https://filmzie.com/content/dont-call-me-bigfo...
4,Forgotten Treasures of Egypt,English,English,2022,91,Documentary,The forgotten treasures of Egypt in the center...,Pelicula,https://filmzie.com/content/forgotten-treasure...


In [7]:
peliculas.to_csv(f"{ruta_base}Peliculas.csv",index=False)

In [8]:
peliculas_dummies = generador_dummies(peliculas)

In [9]:
# Exportamos base de datos de Peliculas con dummies
peliculas_dummies.to_csv(f"{ruta_base}Peliculas_dummies.csv",index=False)

peliculas_dummies.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,sinopsis,tipo,link,Crime,Animation,...,Comedy,Musical,War,News,Biography,Romance,Mystery,Fantasy,History,Horror
0,Inappropriate,Italian,English,2020,23,Experience the extraordinary love between two ...,Pelicula,https://filmzie.com/content/inappropriate-2020,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Peace Pox,English,English,2011,67,"Inspired by the Christmas Truce of 1914, soldi...",Pelicula,https://filmzie.com/content/peace-pox-2011,0,0,...,0,0,1,0,0,0,0,0,1,0
2,The Dark Web,English,"English, Spanish, Albanian",2020,27,How far will a father go to save his son?,Pelicula,https://filmzie.com/content/the-dark-web-2020,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Don't Call Me Bigfoot,English,"English, Spanish",2020,72,"There are tales all around the world of big, h...",Pelicula,https://filmzie.com/content/dont-call-me-bigfo...,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Forgotten Treasures of Egypt,English,English,2022,91,The forgotten treasures of Egypt in the center...,Pelicula,https://filmzie.com/content/forgotten-treasure...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
series.drop(columns=['duracion_minutos'], inplace=True)
series

,titulo,idioma,subtitulos,ano,categorias,sinopsis,tipo,link
0,We Are All Human,English,English,2020,"Documentary, Drama",A collection of short films that evolve around...,Serie,https://filmzie.com/content/we-are-all-human-2020
1,Tough Rides,English,English,2013,"Adventure, Action, Reality TV",Colin Pyle and Ryan Pyle have just completed a...,Serie,https://filmzie.com/content/tough-rides-2013
2,Conquering Northern China,English,English,2017,Documentary,Follow vloggers SerpentZA and Laowhy86 on a pu...,Serie,https://filmzie.com/content/conquering-norther...
3,Vegan Mashup,English,English,2012,"Documentary, Family, Reality TV","Indulge in the culinary delights of ""Vegan Mas...",Serie,https://filmzie.com/content/vegan-mashup-2012
4,Slope Slap,English,English,2019,Reality TV,Slope Slap is a big event close ended reality ...,Serie,https://filmzie.com/content/slope-slap-2019
...,...,...,...,...,...,...,...,...
94,The Shadow Dimension,English,English,2021,Documentary,"A researcher investigates shadow phenomena, se...",Serie,https://filmzie.com/content/the-shadow-dimensi...
95,"Wow, I Never Knew That!",English,English,2011,History,Discover the intriguing origins of everyday th...,Serie,https://filmzie.com/content/wow-i-never-knew-t...
96,The Beech Boys,English,English,2019,Comedy,An egomaniac and his dimwitted best friend mak...,Serie,https://filmzie.com/content/the-beech-boys-2019
97,Dark Passages,English,English,2021,"Documentary, Fantasy, History","Exploring lesser-known conspiracy theories, th...",Serie,https://filmzie.com/content/dark-passages-2021


In [11]:
links_series = list(series["link"].values)
links_series

['https://filmzie.com/content/we-are-all-human-2020',
 'https://filmzie.com/content/tough-rides-2013',
 'https://filmzie.com/content/conquering-northern-china-2017',
 'https://filmzie.com/content/vegan-mashup-2012',
 'https://filmzie.com/content/slope-slap-2019',
 'https://filmzie.com/content/ufo-events-the-best-of-third-phase-2020',
 'https://filmzie.com/content/inner-worlds-the-series-2012',
 'https://filmzie.com/content/scenic-nature-seasons-2023',
 'https://filmzie.com/content/chews-me-2022',
 'https://filmzie.com/content/combat-report-2015',
 'https://filmzie.com/content/ring-of-champions-2019',
 'https://filmzie.com/content/bill-tilghman-and-the-outlaws-2019',
 'https://filmzie.com/content/37-problems-2015',
 'https://filmzie.com/content/pasta-at-home-2020',
 'https://filmzie.com/content/the-special-seer-2019',
 'https://filmzie.com/content/a-greek-islands-destination-cooking-class-santorini-greece-2007',
 'https://filmzie.com/content/bommi--friends-2016',
 'https://filmzie.com/c

In [12]:
procesar = links_series

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {"titulo": [], "temporadas": [], "capitulos": [], "sinopsis" : [], "duracion_minutos" : []}
# diccionario = {"titulo" : alm_3[0], "temporada" : alm_3[1], "capitulos" : alm_3[2], "sinopsis" : alm_3[3], "duracion_minutos" : alm_3[4]}
series_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados_temporales = list(executor.map(series_informacion, procesar))

# Separar los resultados temporales en listas individuales
titulos, temporadas, capitulos, sinopsis, duracion_minutos = zip(*resultados_temporales)

# Agregar los resultados a los diccionarios
resultados_dict["titulo"].extend(titulos)
resultados_dict["temporadas"].extend(temporadas)
resultados_dict["capitulos"].extend(capitulos)
resultados_dict["sinopsis"].extend(sinopsis)
resultados_dict["duracion_minutos"].extend(duracion_minutos)

series_fin = time.time()

print_process(f"La extraccion total de informacion de cada serie tuvo una demora de {round((series_fin-series_comienzo)/ 60, 2) } minutos")

Tiempo total de Combat Report: 29.1 segundos
Tiempo total de We Are All Human: 29.18 segundos
Tiempo total de Inner Worlds: The Series: 29.24 segundos
Tiempo total de Scenic Nature: Seasons: 29.36 segundos
Tiempo total de Slope Slap: 29.44 segundos
Tiempo total de Conquering Northern China: 29.55 segundos
Tiempo total de Chews Me: 29.58 segundos
Tiempo total de Ring Of Champions: 30.01 segundos
Tiempo total de Vegan Mashup: 40.76 segundos
Tiempo total de Tough Rides: 40.84 segundos
Tiempo total de UFO Events: The Best of Third Phase: 46.95 segundos
Tiempo total de Bill Tilghman and the Outlaws: 48.39 segundos
Tiempo total de Pasta at Home: 21.85 segundos
Tiempo total de A Greek Islands Destination Cooking Class (Santorini, Greece): 22.68 segundos
Tiempo total de The Benza: 22.6 segundos
Tiempo total de I Love Christmas Parties: 23.67 segundos
Tiempo total de Threesome: 23.75 segundos
Tiempo total de Bommi & Friends: 24.61 segundos
Tiempo total de 37 Problems: 26.46 segundos
Tiempo tota

In [13]:
df_2 = pd.DataFrame(desempaquetado(resultados_dict))
df_2.head()

,titulo,temporadas,capitulos,sinopsis,duracion_minutos
0,We Are All Human,Season 1,1. Last Statement,Last Statement portrays a series of offenders ...,31
1,We Are All Human,Season 1,2. The Glass Man,"From the depressed and dependent disabled boy,...",20
2,We Are All Human,Season 1,3. Girl-Hearted,It's quite plain to 7 year old Nori: She is a ...,29
3,We Are All Human,Season 1,4. Cambodian Survivor - The Cat,"In Oct 2016, American ﬁlmmaker Antony DeGennar...",24
4,Tough Rides,Season 1: China,1. Shanghai to the North Korean Border,Colin and Ryan Pyle take off from Shanghai on ...,26


In [15]:
series_descripcion = pd.merge(series,df_2, on="titulo", how="inner")
series_descripcion.head()

,titulo,idioma,subtitulos,ano,categorias,sinopsis_x,tipo,link,temporadas,capitulos,sinopsis_y,duracion_minutos
0,We Are All Human,English,English,2020,"Documentary, Drama",A collection of short films that evolve around...,Serie,https://filmzie.com/content/we-are-all-human-2020,Season 1,1. Last Statement,Last Statement portrays a series of offenders ...,31
1,We Are All Human,English,English,2020,"Documentary, Drama",A collection of short films that evolve around...,Serie,https://filmzie.com/content/we-are-all-human-2020,Season 1,2. The Glass Man,"From the depressed and dependent disabled boy,...",20
2,We Are All Human,English,English,2020,"Documentary, Drama",A collection of short films that evolve around...,Serie,https://filmzie.com/content/we-are-all-human-2020,Season 1,3. Girl-Hearted,It's quite plain to 7 year old Nori: She is a ...,29
3,We Are All Human,English,English,2020,"Documentary, Drama",A collection of short films that evolve around...,Serie,https://filmzie.com/content/we-are-all-human-2020,Season 1,4. Cambodian Survivor - The Cat,"In Oct 2016, American ﬁlmmaker Antony DeGennar...",24
4,Tough Rides,English,English,2013,"Adventure, Action, Reality TV",Colin Pyle and Ryan Pyle have just completed a...,Serie,https://filmzie.com/content/tough-rides-2013,Season 1: China,1. Shanghai to the North Korean Border,Colin and Ryan Pyle take off from Shanghai on ...,26
